In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
import os
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import tarfile
import zipfile
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from scipy.stats import chi2_contingency
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [2]:
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import tarfile
import zipfile
from scipy.io import loadmat
from urllib.request import urlretrieve
from os.path import isfile, isdir
import plotly.express as px
from collections import Counter
from scipy.stats import chi2_contingency
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn import metrics
%matplotlib inline

In [3]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
category = df_train['categoria']

In [29]:
tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,  # Limita el número de características
    stop_words='english',  # Elimina stopwords en inglés
    ngram_range=(1, 2),  # Utiliza bigramas además de unigramas
    sublinear_tf=True,  # Aplica sublinear TF scaling
)

In [5]:
# Obtener una lista de todos los títulos
titulos = df_train['titulo'].tolist()

# Crear una matriz (DataFrame) con palabras únicas como columnas
palabrasSeparadas = list(set(' '.join(titulos).split()))
matriz_frecuencia = pd.DataFrame(0, columns=palabrasSeparadas, index=titulos)

# Bucle para contar la frecuencia de palabras en cada título
for titulo in titulos:
    palabras = titulo.split()
    for palabra in palabras:
        matriz_frecuencia.at[titulo, palabra] += 1

In [6]:
todos_titulos = ' '.join(titulos)
palabras = todos_titulos.split()
contador_palabras = Counter(palabras)
palabras_mas_repetidas = contador_palabras.most_common(10)
for palabra, frecuencia in palabras_mas_repetidas:
    print(f'Palabra: {palabra}, Frecuencia: {frecuencia}')

Palabra: -, Frecuencia: 950
Palabra: de, Frecuencia: 296
Palabra: que, Frecuencia: 207
Palabra: vemos, Frecuencia: 205
Palabra: lo, Frecuencia: 203
Palabra: Todo, Frecuencia: 200
Palabra: en, Frecuencia: 66
Palabra: Televidente, Frecuencia: 60
Palabra: la, Frecuencia: 58
Palabra: y, Frecuencia: 57


In [9]:
deleteWord = ["-", "de", "que", "vemos", "lo", "todo", "en", "Televidente", "la", "y"]

def procesar_titulo(titulo):
    # Eliminar y reemplazar palabras específicas 
    palabras = titulo.split()
    filterWord = [palabra for palabra in palabras if palabra not in deleteWord]
    titulo_modificado = ' '.join(filterWord)
    titulo_modificado = titulo_modificado.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    titulo_modificado = titulo_modificado.replace('Á', 'A').replace('É', 'E').replace('Í', 'I').replace('Ó', 'O').replace('Ú', 'U')
    titulo_modificado = titulo_modificado.replace('ñ', 'n').replace('Ñ', 'N')
    
    return titulo_modificado

# Aplicar la función para procesar el título a la columna 'titulo'
df_train['titulo_procesado'] = df_train['titulo'].apply(procesar_titulo)

In [13]:
contingency_table = pd.crosstab(df_train['titulo_procesado'], df_train['categoria'])

correlaciones = []

# Calcular el coeficiente de correlación de Cramer-V para cada título y categoría
for titulo in df_train['titulo_procesado'].unique():
    if titulo in contingency_table.index:
        observed = contingency_table.loc[titulo].to_numpy().astype(np.float64)  # Asegurarse de que observed sea de tipo float64
        observed += 0.5  # Sumar 0.5
        chi2, _, _, _ = chi2_contingency([observed])
        n = observed.sum()
        phi2 = chi2 / n
        phi2corr = max(0, phi2 - 1 / n)
        cramer_v = np.sqrt(phi2corr)
    
        correlaciones.append((titulo, cramer_v))

In [14]:
titulos = df_train['titulo_procesado'].tolist()
matriz_frecuencia = pd.DataFrame(0, columns=palabrasSeparadas, index=titulos)

for titulo in titulos:
    palabras = titulo.split()
    total_palabras = len(palabras)
    
    for palabra in palabras:
        frecuencia = palabras.count(palabra)
        frecuencia_relativa = frecuencia / total_palabras
        matriz_frecuencia.at[titulo, palabra] = frecuencia_relativa

C:\Users\MELASNIE\AppData\Local\Temp\ipykernel_2992\1090353680.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  matriz_frecuencia.at[titulo, palabra] = frecuencia_relativa
C:\Users\MELASNIE\AppData\Local\Temp\ipykernel_2992\1090353680.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  matriz_frecuencia.at[titulo, palabra] = frecuencia_relativa
C:\Users\MELASNIE\AppData\Local\Temp\ipykernel_2992\1090353680.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

In [17]:
def calcular_numero_documentos_con_palabra(data, columna_texto, palabras_unicas):
    num_documentos_con_palabra = []
    for palabra in palabras_unicas:
        num_documentos = sum(data[columna_texto].str.contains(r'\b{}\b'.format(palabra), case=False, regex=False))
        num_documentos_con_palabra.append(num_documentos)
    return num_documentos_con_palabra

# Calcular el IDF para cada palabra
def calcular_idf(total_documentos, num_documentos_con_palabra):
    idf = [np.log(total_documentos / (num + 1)) for num in num_documentos_con_palabra]
    return idf

palabrasSeparadas = list(set(' '.join(df_train['titulo_procesado']).split()))

# Calcular el número total de documentos (títulos)
total_documentos = len(df_train)

# Calcular el número de documentos que contienen cada palabra
num_documentos_con_palabra = calcular_numero_documentos_con_palabra(df_train, 'titulo', palabrasSeparadas)

# Calcular el IDF para cada palabra
idf = calcular_idf(total_documentos, num_documentos_con_palabra)

In [19]:
idf_df = pd.DataFrame({'Palabra': palabrasSeparadas, 'IDF': idf})
idf_df = idf_df[:800]
dataframe = pd.DataFrame(idf_df['IDF'])
print(dataframe)

          IDF
0    6.684612
1    6.684612
2    6.684612
3    6.684612
4    6.684612
..        ...
795  6.684612
796  6.684612
797  6.684612
798  6.684612
799  6.684612

[800 rows x 1 columns]


In [20]:
x_train, x_test, y_train, y_test = train_test_split(dataframe, category, train_size = 0.8, random_state = 42)

In [22]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train, y_train)

RandomForestClassifier()

In [32]:
y_predict = rf.predict(x_test)
cm = confusion_matrix(y_test, y_predict)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             )

In [34]:
prediction = rf.predict(x_test)

In [36]:
df_rl = pd.DataFrame(data= zip(df_test,prediction),
                                       columns=["ID","categoria"])
df_rl.to_csv("./prediction_RandomF.csv",index=False)